# **English - Hindi:**

### **Library Installs:**

In [1]:
pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.5 MB/s eta 0:00:00


### **Imports:**

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import json
import numpy as np
import re
import string
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import math
from tqdm import tqdm
import csv
from torch.optim.lr_scheduler import ReduceLROnPlateau

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### **Data Loading from JSON files:**

In [3]:
TRAIN_PATH = 'train_data1.json'
TEST_PATH = 'test_data1_final.json'

with open(TRAIN_PATH, 'r', encoding='utf-8') as file:
    data = json.load(file)

source_sentences_train = []
target_sentences_train = []

source_sentences_val = []
target_sentences_val = []

id_train = []
id_val = []

for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              target = entry_data["target"]
              if (data_type == "Validation"):
                source_sentences_val.append(source)
                target_sentences_val.append(target)
                id_val.append(entry_id)
              else:
                source_sentences_train.append(source)
                target_sentences_train.append(target)
                id_train.append(entry_id)

with open(TEST_PATH, 'r', encoding='utf-8') as file:
    data = json.load(file)

for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"  Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              if (data_type == "Test"):
                source_sentences_val.append(source)
                id_val.append(entry_id)

Language Pair: English-Hindi
  Data Type: Train
Language Pair: English-Hindi
  Data Type: Test


### **Data PreProcessing:**

In [6]:
def preprocess_and_remove_punctuation(sentence):
    sentence = ''.join([char for char in sentence if char not in string.punctuation and not char.isdigit()])
    return sentence

def preprocess_english(sentences):
    tokenized_sentences = [nltk.word_tokenize(preprocess_and_remove_punctuation(sentence.lower())) for sentence in sentences]
    return tokenized_sentences

def preprocess_hindi(sentences):
    normalizer = IndicNormalizerFactory().get_normalizer("hi")
    processed_sentences = []
    for sentence in sentences:
        normalized_sentence = normalizer.normalize(sentence)
        tokens = list(indic_tokenize.trivial_tokenize(normalized_sentence))
        processed_sentences.append(tokens)
    return processed_sentences

target_sentences_train = [re.sub(r'[a-zA-Z]', '', hi) for hi in target_sentences_train]

english_tokens = preprocess_english(source_sentences_train)
english_test = preprocess_english(source_sentences_val)
hindi_tokens = preprocess_hindi(target_sentences_train)
hindi_test = preprocess_hindi(target_sentences_val)

en_train = english_tokens
en_test = english_test
de_train = hindi_tokens
de_test = hindi_test

en_index2word = ["<PAD>", "<SOS>", "<EOS>"]
de_index2word = ["<PAD>", "<SOS>", "<EOS>"]
en_vocab = set(en_index2word)
de_vocab = set(de_index2word)
for ds in [en_train, en_test]:
    for sent in ds:
        en_vocab.update(sent)
for ds in [de_train, de_test]:
    for sent in ds:
        de_vocab.update(sent)

en_index2word = list(en_vocab)
de_index2word = list(de_vocab)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
en_word2index = {token: idx for idx, token in enumerate(en_index2word)}
de_word2index = {token: idx for idx, token in enumerate(de_index2word)}


def pad_sequence(seq, vocab, max_len):
    padded_seq = [vocab["<SOS>"]] + [vocab.get(word, vocab["<PAD>"]) for word in seq[:max_len - 2]] + [vocab["<EOS>"]]
    padded_seq += [vocab["<PAD>"]] * (max_len - len(padded_seq))
    return padded_seq[:max_len]

class NMTDataset(Dataset):
    def __init__(self, source, target, source_vocab, target_vocab, max_len):
        self.source = [pad_sequence(sent, source_vocab, max_len) for sent in source]
        self.target = [pad_sequence(sent, target_vocab, max_len) for sent in target]

    def __len__(self):
        return len(self.source)

    def __getitem__(self, idx):
        return torch.tensor(self.source[idx]), torch.tensor(self.target[idx])

BATCH_SIZE = 64
MAX_SEQ_LEN = 25
train_src, val_src, train_tgt, val_tgt = train_test_split(en_train, de_train, test_size=0.1, random_state=42)

train_dataset = NMTDataset(train_src, train_tgt, en_word2index, de_word2index, MAX_SEQ_LEN)
val_dataset = NMTDataset(val_src, val_tgt, en_word2index, de_word2index, MAX_SEQ_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


### **Model Architecture:**

In [7]:
# This class adds positional information to input using sinusoidal patterns to
# capture sequence order.
class PositionalEncodingSinusoidal(nn.Module):
    def __init__(self, embedding_dimension, maximum_sequence_length):
        super(PositionalEncodingSinusoidal, self).__init__()
        positions = torch.arange(0, maximum_sequence_length).unsqueeze(1).float()
        division_term = torch.exp(torch.arange(0, embedding_dimension, 2).float() * -(math.log(10000.0) / embedding_dimension))
        positional_embeddings = torch.zeros(maximum_sequence_length, embedding_dimension)
        positional_embeddings[:, 0::2] = torch.sin(positions * division_term) #Use sine for even indices (0, 2, 4, ...)
        positional_embeddings[:, 1::2] = torch.cos(positions * division_term) #Using cosine for odd indices (1, 3, 5, ...).
        self.register_buffer('positional_embeddings', positional_embeddings)

    def forward(self, input_tensor):
        sequence_length = input_tensor.size(1)
        return input_tensor + self.positional_embeddings[:sequence_length, :].unsqueeze(0).to(input_tensor.device)

class NeuralTransformer(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, embedding_dimension, attention_heads, encoder_layers, decoder_layers, dropout_rate, maximum_sequence_length):
        super(NeuralTransformer, self).__init__()
        self.source_embedding = nn.Embedding(source_vocab_size, embedding_dimension)
        self.target_embedding = nn.Embedding(target_vocab_size, embedding_dimension)
        self.positional_encoding = PositionalEncodingSinusoidal(embedding_dimension, maximum_sequence_length)

        self.transformer_core = nn.Transformer(
            d_model=embedding_dimension,
            nhead=attention_heads,
            num_encoder_layers=encoder_layers,
            num_decoder_layers=decoder_layers,
            dropout=dropout_rate,
            batch_first=True
        )
        #Model Output predictions via a linear layer.
        self.output_layer = nn.Linear(embedding_dimension, target_vocab_size)

    def forward(self, source_sequence, target_sequence):
        embedded_source = self.positional_encoding(self.source_embedding(source_sequence))
        embedded_target = self.positional_encoding(self.target_embedding(target_sequence))
        source_mask = self.transformer_core.generate_square_subsequent_mask(source_sequence.size(1)).to(device)
        target_mask = self.transformer_core.generate_square_subsequent_mask(target_sequence.size(1)).to(device)
        transformer_output = self.transformer_core(embedded_source, embedded_target, src_mask=source_mask, tgt_mask=target_mask)
        return self.output_layer(transformer_output)



### **Training and Evaluating the model:**

In [14]:
def sentence_translation(input_sentence, trained_model, source_vocabulary, target_vocabulary, maximum_length):
    trained_model.eval()
    tokenized_input = pad_sequence(input_sentence, source_vocabulary, maximum_length)
    source_tensor = torch.tensor(tokenized_input).unsqueeze(0).to(device)
    target_tokens = [target_vocabulary["<SOS>"]]
    for _ in range(maximum_length):
        target_tensor = torch.tensor(target_tokens).unsqueeze(0).to(device)
        with torch.no_grad():
            predicted_output = trained_model(source_tensor, target_tensor)
        next_predicted_token = predicted_output.argmax(2)[:, -1].item()
        target_tokens.append(next_predicted_token)
        if next_predicted_token == target_vocabulary["<EOS>"]:
            break
    return [list(target_vocabulary.keys())[tok] for tok in target_tokens[1:-1]]


def validate_model(trained_model, validation_data_loader, loss_criterion):
    trained_model.eval()
    total_validation_loss = 0
    with torch.no_grad():
        for src_batch, tgt_batch in validation_data_loader:
            src_batch, tgt_batch = src_batch.to(device), tgt_batch.to(device)
            tgt_inputs = tgt_batch[:, :-1]
            tgt_outputs = tgt_batch[:, 1:]
            with torch.cuda.amp.autocast():
                predicted_outputs = trained_model(src_batch, tgt_inputs)
                reshaped_outputs = predicted_outputs.reshape(-1, predicted_outputs.shape[-1])
                reshaped_targets = tgt_outputs.reshape(-1)
                batch_loss = loss_criterion(reshaped_outputs, reshaped_targets)
            total_validation_loss += batch_loss.item()
    return total_validation_loss / len(validation_data_loader)


def train_transformer_model(model,training_data_loader,validation_data_loader,optimizer,scheduler,loss_criterion,num_epochs, device,source_vocab, target_vocab, max_sequence_length,):
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(num_epochs):
        model.train()
        total_training_loss = 0
        progress_bar = tqdm(enumerate(training_data_loader),total=len(training_data_loader),desc=f"Epoch {epoch + 1}/{num_epochs}",unit="batch" )
        for batch_index, (source_batch, target_batch) in progress_bar:
            source_batch, target_batch = source_batch.to(device), target_batch.to(device)
            target_inputs = target_batch[:, :-1]
            target_outputs = target_batch[:, 1:]
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                model_predictions = model(source_batch, target_inputs)
                reshaped_predictions = model_predictions.reshape(-1, model_predictions.shape[-1])
                reshaped_targets = target_outputs.reshape(-1)
                batch_loss = loss_criterion(reshaped_predictions, reshaped_targets)
            scaler.scale(batch_loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_training_loss += batch_loss.item()
            progress_bar.set_postfix(loss=batch_loss.item())
            if batch_index % 100 == 0:
                test_sentence = ["hello", "brother", "how", "are", "you"]
                translated_sentence = sentence_translation(
                    test_sentence, model, source_vocab, target_vocab, max_sequence_length
                )
                print(f"Batch {batch_index} Translation: {' '.join(test_sentence)} -> {' '.join(translated_sentence)}")
        average_training_loss = total_training_loss / len(training_data_loader)
        print(f"Epoch {epoch + 1} Training Loss: {average_training_loss:.4f}")
        validation_loss = validate_model(model, validation_data_loader, loss_criterion)
        print(f"Epoch {epoch + 1} Validation Loss: {validation_loss:.4f}")
        scheduler.step(validation_loss)
    print("Training complete.")


BATCH_SIZE = 64
MAX_SEQ_LEN = 25
EMBED_DIM = 512
NUM_HEADS = 8
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
DROPOUT = 0.3
LEARNING_RATE = 0.0001
NUM_EPOCHS = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NeuralTransformer(
    source_vocab_size=len(en_word2index),
    target_vocab_size=len(de_word2index),
    embedding_dimension=EMBED_DIM,
    attention_heads=NUM_HEADS,
    encoder_layers=NUM_ENCODER_LAYERS,
    decoder_layers=NUM_DECODER_LAYERS,
    dropout_rate=DROPOUT,
    maximum_sequence_length=MAX_SEQ_LEN
).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=de_word2index["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

train_transformer_model(
    model=model,
    training_data_loader=train_loader,
    validation_data_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    loss_criterion=criterion,
    num_epochs=NUM_EPOCHS,
    device=device,
    source_vocab=en_word2index,
    target_vocab=de_word2index,
    max_sequence_length=MAX_SEQ_LEN,
)

random_sentence = en_test[0]
translated_sentence = sentence_translation(random_sentence, model, en_word2index, de_word2index, MAX_SEQ_LEN)
print(f"Source: {' '.join(random_sentence)}")
print(f"Translation: {' '.join(translated_sentence)}")

<ipython-input-14-f13893bc1e0d>:35: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/10:   0%|          | 0/1137 [00:00<?, ?batch/s]<ipython-input-14-f13893bc1e0d>:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/10:   0%|          | 2/1137 [00:00<03:40,  5.14batch/s, loss=10.6]

Batch 0 Translation: hello brother how are you -> 


Epoch 1/10:   9%|▉         | 102/1137 [00:15<02:42,  6.36batch/s, loss=6.82]

Batch 100 Translation: hello brother how are you -> 


Epoch 1/10:  18%|█▊        | 202/1137 [00:31<03:00,  5.19batch/s, loss=6.73]

...

Batch 1000 Translation: hello brother how are you -> नमस्ते भाई कि आप कैसे हैं ?


Epoch 10/10:  97%|█████████▋| 1102/1137 [02:52<00:06,  5.65batch/s, loss=1.28]

Batch 1100 Translation: hello brother how are you -> नमस्ते कैसे ?


Epoch 10/10: 100%|██████████| 1137/1137 [02:57<00:00,  6.41batch/s, loss=1.49]


Epoch 10 Training Loss: 1.2062
Epoch 10 Validation Loss: 4.1020
Training complete.
Source: and then we need to assure students that a computer can perform all the steps that are used in manual logarithms and so whatever manual algorithm you have we can somehow feed to a computer as well
Translation: और फिर हमें छात्रों को चलाने की आवश्यकता है कि एक कंप्यूटर कंप्यूटर को कंप्यूटर का उपयोग कर सकता है जो उपयोग किया


### **Generating Predictions for Test file:**

In [23]:
TEST_PATH = 'test_data1_final.json'

source_sentences_test = []
id_test = []

with open(TEST_PATH, 'r', encoding='utf-8') as file:
    data = json.load(file)

for language_pair, language_data in data.items():
    if(language_pair == "English-Hindi"):
      print(f"Language Pair: {language_pair}")
      for data_type, data_entries in language_data.items():
          print(f"Data Type: {data_type}")
          for entry_id, entry_data in data_entries.items():
              source = entry_data["source"]
              if (data_type == "Test"):
                source_sentences_test.append(source)
                id_test.append(entry_id)
print(f"Total test sentences: {len(source_sentences_test)}")


OUTPUT_CSV_PATH = "test_translations.csv"

class TestDataset(Dataset):
    def __init__(self, ids, sentences):
        self.ids = ids
        self.sentences = sentences
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, index):
        return self.ids[index], self.sentences[index]

def batch_translate(sentences, model, source_vocab, target_vocab, max_sequence_length):
    tokenized_sentences = [pad_sequence(preprocess_english([sentence])[0], source_vocab, max_sequence_length) for sentence in sentences]
    source_tensor = torch.tensor(tokenized_sentences).to(device)
    batch_size = len(sentences)
    target_tokens = torch.tensor([[target_vocab["<SOS>"]] for _ in range(batch_size)], device=device)
    translated_sentences = [[] for _ in range(batch_size)]

    with torch.set_grad_enabled(False):
        for _ in range(max_sequence_length):
            predicted_output = model(source_tensor, target_tokens)
            next_tokens = predicted_output.argmax(2)[:, -1]
            target_tokens = torch.cat([target_tokens, next_tokens.unsqueeze(1)], dim=1)
            for i, token in enumerate(next_tokens):
                if token.item() != target_vocab["<EOS>"]:
                    translated_sentences[i].append(token.item())
    translated_sentences = [" ".join([list(target_vocab.keys())[tok] for tok in sentence]) for sentence in translated_sentences]
    return translated_sentences

def save_test_translations_to_csv_batch( test_ids, test_sentences, model, source_vocab, target_vocab, max_sequence_length,  output_csv_path, batch_size=32):
    dataset = TestDataset(test_ids, test_sentences)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
    translations = []
    model = model.to(device)

    for batch in tqdm(dataloader, total=len(dataloader), desc="Translating Sentences in Batches"):
        batch_ids, batch_sentences = zip(*batch)
        translated_sentences = batch_translate(
            sentences=batch_sentences,
            model=model,
            source_vocab=source_vocab,
            target_vocab=target_vocab,
            max_sequence_length=max_sequence_length
        )
        translations.extend(zip(batch_ids, translated_sentences))

    with open(output_csv_path, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ID", "Translation"])
        writer.writerows(translations)
    print(f"Translations saved to {output_csv_path}")


save_test_translations_to_csv_batch(
    test_ids=id_test,
    test_sentences=source_sentences_test,
    model=model,
    source_vocab=en_word2index,
    target_vocab=de_word2index,
    max_sequence_length=MAX_SEQ_LEN,
    output_csv_path=OUTPUT_CSV_PATH,
    batch_size=64
)


Language Pair: English-Hindi
Data Type: Test
Total test sentences: 23085


Translating Sentences in Batches: 100%|██████████| 361/361 [25:42<00:00,  4.27s/it]

Translations saved to test_translations.csv


### **Merging Bengali-Hindi Results:**

In [ ]:
import pandas as pd
df1 = pd.read_csv("/content/answersB.csv")
df2= pd.read_csv("/content/answersH.csv")
df3 = pd.concat([df1, df2])

df3.to_csv('answersBH.csv', index = False)
filtered_data = pd.read_csv("/content/answersBH.csv")
answer = "/content/answer.csv"
with open(answer, "w") as f:
  f.writelines("ID\tTranslation\n")
  for i in range(filtered_data.shape[0]):
    f.writelines(f'{filtered_data["ID"][i]}\t"{filtered_data["Translation"][i]}"\n')